<a href="https://colab.research.google.com/github/Southparketeer/3D-Geometry-Deep-Learning-for-Cardiovascular-Metabolic-Risk-Prediction/blob/master/Hyperparameter_Analysis_Bayesian_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning with Bayesian Optimization

## Install Bayesian Optimization Package

In [0]:
!pip install bayesian-optimization

## Tuning Hyperparameter with Bayesian Optimization

In [30]:
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import numpy as np
import pandas as pd

#Load Data
df = pd.read_csv('/content/Label.csv')
y = df['gender'].values
x = np.load('/content/train_3D_geometry_embedded_map.npy')

# Time Function
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# Evaulation Function
def evaluate_network_CNN(dropout, lr_CNN, layer_CNN):
  SPLITS = 5
  boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)
  # Track progress
  mean_benchmark = []
  epochs_needed = []
  num = 0
  batch_size = 128
  num_classes = 1 #Binary  
  img_rows, img_cols = 128, 128 # input image dimensions
  
  # Loop through samples
  for train, test in boot.split(x, y):
    start_time = time.time()
    num += 1
    # Split train and test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    # Setup Model
    model = Sequential()
    
    layer = 0
    while layer < layer_CNN and layer ** 2 < (img_rows/2):
      if layer == 0:
        model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=(2, 2)))
      else:
        model.add(Conv2D(32, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
      layer += 1
      
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr_CNN), metrics=['accuracy'])
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=0, mode='auto', restore_best_weights=True)
    # Train on the bootstrap sample
    model.fit(x = x_train, y = y_train, batch_size=batch_size, epochs=20, verbose=0,validation_data=(x_test, y_test), callbacks=[monitor])
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)
    # Predict on the out of boot (validation)
    pred = model.predict(x_test)
    score = metrics.log_loss(y_test, pred)
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)
    time_took = time.time() - start_time
    #print(f"#{num}: score={score:.6f}, mean score={m1:.6f}, stdev={mdev:.6f}, epochs={epochs}, mean epochs={int(m2)}, time={hms_string(time_took)}")
  tensorflow.keras.backend.clear_session()
  return (-m1)    

#epoch selected based on Tensorboard training performence to pervent overfitting
#print(evaluate_network(dropout = 0.2,lr_CNN = 1e-3,layer_CNN = 3))

from bayes_opt import BayesianOptimization
import time

# Bounded region of parameter space
pbounds = {'dropout': (0.0, 0.499),
           'lr_CNN': (0.0, 0.1),
           'layer_CNN': (1,3),
          }

optimizer = BayesianOptimization(
    f=evaluate_network_CNN,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

start_time = time.time()
optimizer.maximize(init_points=10, n_iter=10)
time_took = time.time() - start_time

print(optimizer.max)

|   iter    |  target   |  dropout  | layer_CNN |  lr_CNN   |
-------------------------------------------------------------
|  1        | -0.3593   |  0.2081   |  2.441    |  1.144e-0 |
|  2        | -0.5978   |  0.1509   |  1.294    |  0.009234 |
|  3        | -0.5999   |  0.09294  |  1.691    |  0.03968  |
|  4        | -0.6279   |  0.2689   |  1.838    |  0.06852  |
|  5        | -0.1936   |  0.102    |  2.756    |  0.002739 |
|  6        | -0.5151   |  0.3346   |  1.835    |  0.05587  |
|  7        | -0.6844   |  0.07005  |  1.396    |  0.08007  |
|  8        | -0.6897   |  0.4832   |  1.627    |  0.06923  |
|  9        | -0.2217   |  0.4373   |  2.789    |  0.008504 |
|  10       | -0.6894   |  0.01949  |  1.34     |  0.08781  |
|  11       | -0.6896   |  0.0      |  3.0      |  0.1      |
|  12       | -0.6968   |  0.499    |  2.241    |  0.0      |
|  13       | -0.6965   |  0.499    |  1.0      |  0.0      |
|  14       | -0.701    |  0.499    |  3.0      |  0.0      |
|  15   

## Hyperparameter Validation

In [35]:
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import numpy as np
import pandas as pd

#Load Data
df = pd.read_csv('/content/Label.csv')
y = df['gender'].values
x = np.load('/content/train_3D_geometry_embedded_map.npy')

# Time Function
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# Evaulation Function
def evaluate_network_CNN(dropout, lr_CNN, layer_CNN):
  SPLITS = 5
  boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)
  # Track progress
  mean_benchmark = []
  epochs_needed = []
  num = 0
  batch_size = 128
  num_classes = 1 #Binary  
  img_rows, img_cols = 128, 128 # input image dimensions
  
  # Loop through samples
  for train, test in boot.split(x, y):
    start_time = time.time()
    num += 1
    # Split train and test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    # Setup Model
    model = Sequential()
    
    layer = 0
    while layer < layer_CNN and layer ** 2 < (img_rows/2):
      if layer == 0:
        model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=(2, 2)))
      else:
        model.add(Conv2D(32, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
      layer += 1
      
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr_CNN), metrics=['accuracy'])
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=0, mode='auto', restore_best_weights=True)
    # Train on the bootstrap sample
    model.fit(x = x_train, y = y_train, batch_size=batch_size, epochs=20, verbose=1,validation_data=(x_test, y_test), callbacks=[monitor])
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)
    # Predict on the out of boot (validation)
    pred = model.predict(x_test)
    score = metrics.log_loss(y_test, pred)
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.6f}, mean score={m1:.6f}, stdev={mdev:.6f}, epochs={epochs}, mean epochs={int(m2)}, time={hms_string(time_took)}")
  tensorflow.keras.backend.clear_session()
  return (-m1)    

print(evaluate_network_CNN(dropout = 0.1020216726160272,lr_CNN = 0.0027387593197926163,layer_CNN = 3))

Train on 1800 samples, validate on 200 samples
Epoch 1/20
1800/1800 [==============================] - 1s 659us/sample - loss: 0.6831 - acc: 0.6483 - val_loss: 0.4481 - val_acc: 0.8000
Epoch 2/20
1800/1800 [==============================] - 1s 358us/sample - loss: 0.3511 - acc: 0.8567 - val_loss: 0.3111 - val_acc: 0.8850
Epoch 3/20
1800/1800 [==============================] - 1s 356us/sample - loss: 0.2709 - acc: 0.9033 - val_loss: 0.2615 - val_acc: 0.8800
Epoch 4/20
1800/1800 [==============================] - 1s 354us/sample - loss: 0.2277 - acc: 0.9106 - val_loss: 0.2975 - val_acc: 0.9150
Epoch 5/20
1800/1800 [==============================] - 1s 353us/sample - loss: 0.2680 - acc: 0.9022 - val_loss: 0.2671 - val_acc: 0.9100
Epoch 6/20
1800/1800 [==============================] - 1s 354us/sample - loss: 0.2440 - acc: 0.9000 - val_loss: 0.2518 - val_acc: 0.8950
Epoch 7/20
1800/1800 [==============================] - 1s 358us/sample - loss: 0.2171 - acc: 0.9172 - val_loss: 0.2566 - val